In [1]:
import gc
import glob
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from random import randrange
from tqdm import tqdm, tnrange
from tqdm.notebook import tqdm_notebook

import warnings

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from keras.optimizer_v2.gradient_descent import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Lambda, Dropout, LeakyReLU, BatchNormalization, Activation, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from mlxtend.evaluate import accuracy

from tensorflow.keras.optimizers import Adam,RMSprop,SGD, Nadam
#from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.applications import VGG16

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # ignore tensorflow warnings
warnings.filterwarnings("ignore")

In [3]:
path_data = "data/data_palm_vein/NIR"

In [7]:
def load_img(path, xdim=128, ydim=128, nb_of_users = 500):
    label_names = []
    X = []
    y = []
    nb_of_users = nb_of_users - 1
    count = 0
    identity = -1
    directories = os.listdir(path)
    directories.sort()
    for dirname in tqdm_notebook(directories, desc="Loading images..."):
        if dirname == ".DS_Store": continue
        label_names.append(dirname)
        data_path = os.path.join(path + "/" + dirname, '*g')
        files = glob.glob(data_path)
        if identity >= nb_of_users: break
        identity += 1
        files.sort()
        for f1 in files:
            img = cv2.imread(f1, cv2.IMREAD_GRAYSCALE)
            #img = cv2.imread(f1)
            #img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
            img = cv2.resize(img,(int(xdim*1), int(ydim*1)))
            #X.append([np.array(img), np.array(img), np.array(img)])
            #X.append(np.array(img))
            #X.append(np.array(img))
            X.append(np.array(img))
            #stacked_img = np.stack((img,)*3, axis=-1)
            #X.append(stacked_img)
            y.append(identity)
            count += 1
    X = np.array(X)
    y = np.array(y)
    print("\n ================= Summary of extraction ================= \n")
    print(count, ' images lues')
    print("\nX.shape = ", X.shape)
    print("y.shape = ", y.shape)
    gc.collect()
    return X, y, label_names

In [8]:
X, y, label_names = load_img(path_data, nb_of_users=50, xdim=224, ydim=224)

Loading images...:   0%|          | 0/501 [00:00<?, ?it/s]


 ================= Summary of extraction ================= 

600  images lues

X.shape =  (600, 224, 224)
y.shape =  (600,)


In [9]:
X = X / 255.
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y = to_categorical(y)

print("\nPREPROCESSING DATA")
print("-----------------------------------------")
print("\nX shape : {}".format(X.shape))
print("-----------------------------------------")
print("y shape : {}\n".format(y.shape))
print("\n\n-----------------------------------------")
print("Il y a {} utilisateur(s) dans le dataset prélevé.".format(y.shape[1]))


PREPROCESSING DATA
-----------------------------------------

X shape : (600, 224, 224, 1)
-----------------------------------------
y shape : (600, 50)



-----------------------------------------
Il y a 50 utilisateur(s) dans le dataset prélevé.


In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nSplitting data ...\n")
print("\n=============================== Splitting data =============================== \n")
print("\nX_train shape : {}    |   y_train shape : {}".format(X_train.shape, y_train.shape))
print("------------------------------------------------------------------------------")
print("(X_temp shape : {}    |   y_temp shape  : {})\n".format(X_temp.shape, y_temp.shape))


Splitting data ...


=============================== Splitting data =============================== 


X_train shape : (480, 224, 224, 1)    |   y_train shape : (480, 50)
------------------------------------------------------------------------------
(X_temp shape : (120, 224, 224, 1)    |   y_temp shape  : (120, 50))



In [15]:
def zfnet_model(input_shape=(128, 128, 3), nombre_classes=500):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(7, 7), strides=(2, 2), padding="valid", activation="LeakyReLU",
                     kernel_initializer="uniform", input_shape=input_shape))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # model.add(Lambda(lambda x: tf.image.per_image_standardization(x)))

    model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2), padding="same",
                     activation="LeakyReLU", kernel_initializer="uniform"))

    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    # model.add(Lambda(lambda x: tf.image.per_image_standardization(x)))

    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding="same",
                     kernel_initializer="uniform"))

    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding="same",
                     kernel_initializer="uniform"))
    model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding="same",
                     activation="LeakyReLU", kernel_initializer="uniform"))

    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, activation="LeakyReLU"))
    model.add(Dropout(0.5))
    model.add(Dense(units=4096, activation="LeakyReLU"))
    model.add(Dropout(0.5))
    model.add(Dense(units=nombre_classes, activation="softmax"))

    print(model.summary())
    print("\n ================= ZFNET model ================= \n")


    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.7, 
                                            min_lr=0.00000000001)
#    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
#                                            factor=0.1, patience=1, min_lr=0.00001)


    return model, learning_rate_reduction

In [16]:
Nombre_classes = y.shape[1]
input_shape = (X.shape[1], X.shape[2], 1)
print("\nNombres de classes : {}   |   Input shape : {}\n".format(Nombre_classes, input_shape))


Nombres de classes : 50   |   Input shape : (224, 224, 1)



In [17]:
model, learning_rate_reduction = zfnet_model(input_shape=input_shape, nombre_classes=Nombre_classes)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 109, 109, 96)      4800      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 256)      

In [27]:
epochs = 20
batch_size = 32
print("\n ================= Training : ZFNET model ================= \n")
print("  Epochs :  {}   |   Batch size : {} ".format(epochs, batch_size))
print("\n ========================================================== \n")

trained_model = model.fit(X_train, y_train, validation_data=(X_temp, y_temp), epochs=epochs, callbacks=learning_rate_reduction)


 ================= Training : ZFNET model ================= 

  Epochs :  20   |   Batch size : 32 


Epoch 1/20
15/15 [==============================] - 9s 529ms/step - loss: 3.8834 - accuracy: 0.0188 - val_loss: 4.0367 - val_accuracy: 0.0083
Epoch 2/20
15/15 [==============================] - 8s 532ms/step - loss: 3.8845 - accuracy: 0.0292 - val_loss: 4.0372 - val_accuracy: 0.0000e+00
Epoch 3/20
15/15 [==============================] - 8s 525ms/step - loss: 3.8911 - accuracy: 0.0271 - val_loss: 4.0352 - val_accuracy: 0.0083

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.8247522277524694e-05.
Epoch 4/20
15/15 [==============================] - 8s 527ms/step - loss: 3.8874 - accuracy: 0.0125 - val_loss: 4.0356 - val_accuracy: 0.0000e+00
Epoch 5/20
15/15 [==============================] - 8s 536ms/step - loss: 3.8846 - accuracy: 0.0229 - val_loss: 4.0356 - val_accuracy: 0.0083

Epoch 00005: ReduceLROnPlateau reducing learning rate to 1.977326610358432e-05.
Epoch 6/20
15/15

In [29]:
val = model.evaluate(X_temp, y_temp)

print("\n ================= Evaluation : ZFNET model ================= \n")
print("  With : \n")
print("Batch size         :  {}     |   Epochs      : {} ".format(batch_size, epochs))
print("Nombres de classes :  {}     |   Input shape : {} \n".format(Nombre_classes, input_shape))
print("\n ============================================================= \n")

print("  Results : \n")
print("Loss  : %.2f%%" % (val[0] * 100))
print("Score : %.2f%%" % (val[1] * 100))

4/4 [==============================] - 0s 78ms/step - loss: 4.0349 - accuracy: 0.0000e+00

 ================= Evaluation : ZFNET model ================= 

  With : 

Batch size         :  32     |   Epochs      : 20 
Nombres de classes :  50     |   Input shape : (224, 224, 1) 



  Results : 

Loss  : 403.49%
Score : 0.00%
